# ⚡️ QuickStart: VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/quickstart/VideoDB%20Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is designed to help you get started with [VideoDB](https://videodb.io). 

<div style="height:40px;"></div>

### Setup
---  


#### 🔧 Install VideoDB in your environment

VideoDB is available as [python package 📦](https://pypi.org/project/videodb)  

In [1]:
!pip install -U videodb 

##### 🔗 Setting Up a connection to db
To connect to VideoDB, simply create a `Connection` object. 

This can be done by either providing your VideoDB API key directly to the constructor or by setting the `VIDEO_DB_API_KEY` environment variable with your API key. 

>💡
>Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, No credit card required ) 🎉.

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()

<div style="height:40px;"></div>

### Working with a single Video
---

<div style="height:10px;"></div>

#### ⬆️ Upload a video
Now that you have established a connection to VideoDB, you can now upload your videos using `conn.upload()`.

You can upload your media by a public `url` or from your `file_path` 

`upload` returns a `Video` Object, which can be used to access video methods.

In [4]:
#Upload a video by url
video = conn.upload(url="https://www.youtube.com/watch?v=wU0PYcCsL6o")

<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
VideoDB simplifies your upload by supporting links from Youtube, S3 or any Public URL with video
</div>

<div style="height:15px;"></div>

#### 📺 View your video

Your video is instantly available for viewing 720p resolution ⚡️

* Generate a streamable url for video using `video.generate_stream()` 
* Preview the video using Video.play(). This will open the video in your default browser/notebook.

<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
    <strong>Note:</strong>if you are viewing this notebook on github, you won't be able to see iframe player, because of security restrictions. <br> 
    Please open the printed link of player in your browser</div>


In [6]:
video.generate_stream()
video.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/0645b883-5e48-4136-a5bc-f046b3a166a2.m3u8'

#### ✂️ Get Specific Sections of videos

You can easily clip specific sections of a video by passing timeline  of start and end sections. It accepts seconds.   
For example Here’s we are streaming only first 10 seconds and then 120 to 140 second of uploaded video

In [7]:
stream_link = video.generate_stream(timeline=[[0,10], [120,140]]) 
play_stream(stream_link)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/d0a049ff-be6e-41f7-826e-4a6e411f7bab.m3u8'

<div style="height:15px;"></div>

#### 🔍 Indexing  a Video

To search bits inside a video, you  have to first index the video. This can be done by a invoking the index function on the video object. VideoDB offers two type of indexes currently.

1. `index_spoken_words`: Indexes spoken words in the video. It automatically generate the transcript and makes it ready for search.  20+ languages are supported. Checkout [Language Support](https://docs.videodb.io/language-support-79) to know more.

2. `index_scenes`: Indexes visual information and events of the video. Perfect for finding scenes, activities, objects, emotions in the video. Refer [Scene Index Documentation](https://docs.videodb.io/scene-index-documentation-80) for details.
   
<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
    <strong>Note:</strong>Index may take time for longer videos</div>

In [8]:
#index spoken content of the video.
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.40it/s]                                                                           


In [36]:
# index visual information in video frames, You can change the prompt according to your usecase.
# You can index a video multiple times with different prompts.
index_id = video.index_scenes(
    prompt="Describe the scene in strictly 100 words"
)
# Wait to Indexing to finish
scene_index = video.get_scene_index(index_id)
scene_index

[{'description': "This stunning scene captures the majestic beauty of a snow-covered mountain peak piercing through a clear blue sky. The jagged ridgeline glistens under the sunlight, displaying a breathtaking expanse of pure white. Surrounding peaks and ridges stretch across the horizon, some shrouded in mist and clouds, adding depth to the expansive, tranquil landscape. The stark contrast between the bright snow, dark rocky outcrops, and the azure sky creates a dramatic and awe-inspiring view. The serenity and grandeur of this high-altitude environment evoke a sense of wonder and contemplation, showcasing nature's pristine and rugged beauty in its purest form.", 'end': 5.8, 'start': 0.0}, {'description': "A man rides a bicycle on a bustling city street, surrounded by towering high-rise buildings. He's in motion, his posture leaning slightly forward as if to accelerate. Pedestrians walk along the sidewalks, some carrying bags or talking on their phones. The weather appears mild, with 

<div style="height:15px;"></div>

### Search inside a video:  
  
Search can peformed on indexed video using `video.search()` While searching you have options to choose the type of search and index. VideoDB offers following types of search :

`SearchType.semantic`  Perfect for question answer kind of queries. This is also the default type of search. 

`SearchType.keyword`  It matches the exact occurrence of word or sentence you pass in the query parameter of the search function. keyword search is only available to use with single videos. 

`IndexType.scene` It search the visual information of the video, Index the video using  index_scenes  function.

`IndexType.spoken_word` It search the spoken information of the video,  Index the video using  index_spoken_words  function.

In [13]:
from videodb import SearchType, IndexType

result = video.search(query="what's the dream?", search_type=SearchType.semantic, index_type=IndexType.spoken_word)
result.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/de284dc9-1aa9-440b-b407-b8fc5aa07273.m3u8'

In [16]:
# Try with different queries

# "city scene with buses"
query = "mountains"

result = video.search(query=query, search_type=SearchType.semantic, index_type=IndexType.scene)
result.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/72baa6ff-2758-49bb-b8d6-237f44e4b6c1.m3u8'

##### 📺 View Search Results :

`video.search()` will return a SearchResults object, which contains the sections/shots of videos which semantically match your search query

* `result.get_shots()` - Returns a list of Shot that matched search query
* `result.play()`  - This will open the video in your default browser/notebook

##### 🗑️ Cleanup
You can delete the video from database using `video.delete()`

In [60]:
video.delete()

<div style="height:40px;"></div>

## RAG: Working with Multiple Videos
---
`VideoDB` can store and search inside multiple videos with ease.  By default, videos are uploaded to your default collection and you have freedom to create and manage more collections, checkout our [Collections docs](https://docs.videodb.io/collections-68) for more details. 

If you are an existing llamaIndex user, trying to build RAG pipeline on your video data. You can use VideoDB retriever. 
Checkout [llama-Index docs](https://docs.llamaindex.ai/en/stable/examples/retrievers/videodb_retriever.html)

<div style="height:15px;"></div>

##### 🔄 Using Collection to upload multiple Videos

In [61]:
# Get a collection
coll = conn.get_collection()

# Upload Videos to a collection
coll.upload(url="https://www.youtube.com/watch?v=lsODSDmY4CY")
coll.upload(url="https://www.youtube.com/watch?v=vZ4kOr38JhY")
coll.upload(url="https://www.youtube.com/watch?v=uak_dXHh6s4")

Video(id=m-6d1b1bf6-a2ba-4786-a3a8-8efbabfee28c, collection_id=c-a2dc1100-1faa-4394-803c-ca808da09c47, stream_url=https://d27qzqw9ehjjni.cloudfront.net/v3/published/manifests/4e41464d-0ca6-40da-86f7-3e803ff8d840.m3u8, player_url=https://console.videodb.io/player?url=https://d27qzqw9ehjjni.cloudfront.net/v3/published/manifests/4e41464d-0ca6-40da-86f7-3e803ff8d840.m3u8, name=AMA #3: Adaptogens, Fasting & Fertility, Bluetooth/EMF Risks, Cognitive Load Limits & More, description=None, thumbnail_url=None, length=1921.2193)

* `conn.get_collection()` : Returns Collection object, the default collection
* `coll.get_videos()` : Returns list of Video, all videos in collections
* `coll.get_video(video_id)` : Returns Video, respective video object from given video_id
* `coll.delete_video(video_id)` : Deletes the video from Collection

<div style="height:15px;"></div>

### 📂 Search on Multiple Videos from a collection

You can simply Index all the videos in a collection and use search method on collection to find relevant results.   
Here we are indexing spoken content of a collection for quick experiment.

<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
    <strong>Note:</strong> Index may take time for longer videos</div>

In [62]:
# for simplicity we are just indexing the spoken content of each video.
for video in coll.get_videos():
    video.index_spoken_words()
    print(f"Indexed {video.name}")

Indexed AMA #3: Adaptogens, Fasting & Fertility, Bluetooth/EMF Risks, Cognitive Load Limits & More
Indexed AMA #1: Leveraging Ultradian Cycles, How to Protect Your Brain, Seed Oils Examined and More
Indexed AMA #2: Improve Sleep, Reduce Sugar Cravings, Optimal Protein Intake, Stretching Frequency & More


<div style="height:15px;"></div>

### Search Inside Collection:   
  
Search can be peformed on a collection using `coll.search()`

In [66]:
# search in the collection of videos
results = coll.search(query = "Deep sleep")
results.play()

'https://console.videodb.io/player?url=https://d27qzqw9ehjjni.cloudfront.net/v3/published/manifests/fdd420dd-b104-43dd-9a0f-d3f0afd5b718.m3u8'

In [67]:
results = coll.search(query= "What are the benifits of morning sunlight?")
results.play()

'https://console.videodb.io/player?url=https://d27qzqw9ehjjni.cloudfront.net/v3/published/manifests/60cc252f-abf9-497d-93db-0a53dbffdcf0.m3u8'

In [68]:
results = coll.search(query= "What are Adaptogens?")
results.play()

'https://console.videodb.io/player?url=https://d27qzqw9ehjjni.cloudfront.net/v3/published/manifests/9f690de4-758e-4ac8-9316-916a30e01e72.m3u8'

#### 📺 View Search Results :

`video.search()` will return a SearchResults object, which contains the sections/shots of videos which semantically match your search query

* `result.get_shots()` - Returns a list of Shot that matched search query
* `result.play()`  - This will open the video in your default browser/notebook

<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
As you can see VideoDB fundamentally removes the limitation of files and gives you power to access and stream videos in a very seamless way. Stay tuned for exciting features in our upcoming version and keep building awesome stuff with VideoDB 🤘
</div>

<div style="height:40px;"></div>

### 🌟 Explore more with Video object
There are multiple methods available on a Video Object, that can be helpful for your use-case.

#### Access Transcript

In [21]:
# words with timestamps
text_json = video.get_transcript()
text = video.get_transcript_text()
print(text)

- If you awaken from this illusion and you understand that black implies white, self implies other, life implies death, or shall I say death implies life, you can feel yourself not as a stranger in the world, not as something here on probation, not as something that has arrived here by fluke, but you can begin to feel your own existence as absolutely fundamental. I'm not trying to sell you on this idea in the sense of converting you to it. I want you to play with it. I want you to think of its possibilities. I'm not trying to prove it. I'm just putting it forward as a possibility of life to think about. So then - let's suppose that you were able every night to dream any dream you wanted to dream and that you could, for example, have the power within one night to dream 75 years of time or any length of time you wanted to have. And you would naturally, as you began on this adventure of dreams, you would fulfill all your wishes. You would have every kind of pleasure conceived. And after s

#### Access Visual Scene Descriptions

In [20]:
# Take a look at the scenes
video.get_scene_index(index_id)

[{'description': 'Snow-covered mountain peaks dominate the scene, shrouded in a blanket of ice and snow. The sharp ridges and steep slopes are breathtaking, with one prominent peak in the foreground. The sun shines brightly, casting shadows that accentuate the rugged terrain. In the distance, a series of jagged summits stretch across the horizon, partially obscured by wisps of clouds. The deep blue sky contrasts starkly with the white snow, creating a dramatic and serene landscape. The vastness and majesty of the mountains evoke a sense of awe, showcasing the raw beauty of the natural world.',
  'end': 5.8,
  'start': 0.0},
 {'description': 'A vibrant, bustling festival scene filled with bright decorations and lively atmosphere. A diverse crowd of people in colorful outfits enjoys the festivities, some dancing, others taking photos or buying food from various stalls. Flags and streamers adorn the area, and a stage features a lively performance. Children run around playfully, adding to 

#### Add Subtitle to a video 
It returns a new stream instantly with subtitle added into the video. Subtitle functions has many styling parameters like font, size, background color etc. Check the notebook: [Subtitle Styles](https://github.com/video-db/videodb-cookbook/blob/main/guides/Subtitle.ipynb) for details.

In [22]:
new_stream = video.add_subtitle()
play_stream(new_stream)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/c4f9baf4-180e-4cdb-b48d-407c71fe4080.m3u8'

#### Generate Thumbnail of Video :

You can use `video.generate_thumbnail(time=)` to generate a thumbnail image of video from any timestamp.

In [34]:
from IPython.display import Image

image = video.generate_thumbnail(time=12.0)
Image(url=image.url) 

##### Delete a video :

* `video.delete()` :deletes a video.

In [ ]:
video.delete()


<div style="background-color: #ffffcc; color: black; padding: 10px; border-radius: 5px;">
Checkout more examples and tutorials 👉 <a href="https://docs.videodb.io/build-with-videodb-35"> Build with VideoDB </a> to explore what you can build with VideoDB
</div>